<a href="https://colab.research.google.com/github/maryamelnahas/CSEN711-Project/blob/main/BINF711_MS1_Password_Strength_Analyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import numpy as np
import math
import os
from google.colab import files
import hashlib
import secrets
import getpass


user_database = {}
attacker_dictionary = {}
MAX_DICT_SIZE = 50000

MAX_FAILED_ATTEMPTS = 3
PASSWORD_EXPIRY_COUNT = 10

L33T_MAP = {
    '@': 'a', '4': 'a',
    '8': 'b',
    '(': 'c',
    '3': 'e',
    '9': 'g', '6': 'g',
    '1': 'i', '!': 'i', '|': 'i',
    '0': 'o',
    '5': 's', '$': 's',
    '7': 't', '+': 't',
    'z': 's'
}

STRENGTH_THRESHOLDS = {
    'very weak': 15,     # < ~32,000 guesses
    'weak': 25,          # < ~33 million guesses
    'medium': 35,        # < ~34 billion guesses
    'strong': 50,        # < ~1 quadrillion guesses
    'very strong': 50    # > ~1 quadrillion guesses
}


def load_ranked_dictionary(file_name, max_size):
    print(f"Loading ranked dictionary from {file_name}...")
    passwords = {}
    rank = 1
    try:
        with open(file_name, 'r', encoding='utf-8', errors='ignore') as f:
            for line in f:
                if rank > max_size:
                    break
                password = line.strip().split('\t')[0].lower()
                if password and password not in passwords:
                    passwords[password] = rank
                    rank += 1
        print(f"Loaded {len(passwords)} unique passwords into ranked dictionary.")
        return passwords
    except FileNotFoundError:
        print(f"Error: File '{file_name}' not found. Using a small placeholder dictionary.")
        return {"password": 1, "123456": 2, "qwerty": 3, "secret": 4, "iloveyou": 5}


def demangle_l33t(password):
    password_lower = password.lower()
    demangled = "".join(L33T_MAP.get(char, char) for char in password_lower)
    return demangled

def get_brute_force_entropy(password):
    charset_size = 0
    if any(c.islower() for c in password): charset_size += 26
    if any(c.isupper() for c in password): charset_size += 26
    if any(c.isdigit() for c in password): charset_size += 10
    if any(c in "!@#$%^&*()_+-=[]{}|;:,.<>?~" for c in password): charset_size += 32
    if charset_size == 0: return 1.0
    return float(charset_size ** len(password))


def calculate_password_complexity(password, dictionary):
    if not password: return 1.0
    password_lower = password.lower()
    guess_counts = [get_brute_force_entropy(password)]

    # Attack 1: Direct Dictionary
    if password_lower in dictionary:
        guess_counts.append(dictionary[password_lower])
    # Attack 2: Capitalization
    if password_lower in dictionary and password != password_lower:
        guess_counts.append(dictionary[password_lower] * 10)
    # Attack 3: L33t-speak
    demangled_password = demangle_l33t(password)
    if demangled_password in dictionary:
        guess_counts.append(dictionary[demangled_password] * 100)
    # Attack 4: Reversed
    reversed_password = password_lower[::-1]
    if reversed_password in dictionary:
        guess_counts.append(dictionary[reversed_password] * 50)

    return max(1.0, min(guess_counts))


def get_strength_rating(guess_count):
    """Maps the final guess count to a human-readable strength rating."""
    try:
        complexity_bits = math.log2(guess_count)
    except ValueError:
        complexity_bits = 0

    if complexity_bits < STRENGTH_THRESHOLDS['very weak']: return "Very Weak"
    elif complexity_bits < STRENGTH_THRESHOLDS['weak']: return "Weak"
    elif complexity_bits < STRENGTH_THRESHOLDS['medium']: return "Medium"
    elif complexity_bits < STRENGTH_THRESHOLDS['strong']: return "Strong"
    else: return "Very Strong"


def get_password_hash(password, salt):
    return hashlib.pbkdf2_hmac('sha256', password.encode('utf-8'), salt, 100000)

def verify_password(stored_hash, stored_salt, provided_password):
    new_hash = get_password_hash(provided_password, stored_salt)
    return secrets.compare_digest(new_hash, stored_hash)


def sign_up(username, password):

    global user_database, attacker_dictionary

    if username in user_database:
        return f"🚨 Error: Username '{username}' already exists."

    guess_count = calculate_password_complexity(password, attacker_dictionary)
    strength = get_strength_rating(guess_count)

    if strength in ["Very Weak", "Weak"]:
        return f"🚨 Error: Password is too weak ({strength}). Please choose a stronger password."

    salt = os.urandom(16)
    password_hash = get_password_hash(password, salt)

    user_database[username] = {
        "password_hash": password_hash,
        "salt": salt,
        "is_locked": False,
        "failed_attempts": 0,
        "logins_remaining": PASSWORD_EXPIRY_COUNT
    }

    return f"✅ Success: Account '{username}' created. (Strength: {strength})"

def login(username, password):

    global user_database

    if username not in user_database:
        return "Login Failed: Invalid username or password."

    user_data = user_database[username]

    if user_data["is_locked"]:
        return "Login Failed: Account is locked due to too many failed attempts."

    if user_data["logins_remaining"] <= 0:
        return "Login Failed: Password has expired. Please reset."

    if verify_password(user_data["password_hash"], user_data["salt"], password):

        user_data["failed_attempts"] = 0
        user_data["logins_remaining"] -= 1

        return f"✅ Login Successful. You have {user_data['logins_remaining']} logins remaining."

    else:
        # --- FAILURE ---
        user_data["failed_attempts"] += 1

        # Check if this failure triggers a lockout
        if user_data["failed_attempts"] >= MAX_FAILED_ATTEMPTS:
            user_data["is_locked"] = True
            return "Login Failed: Invalid password. Your account is now locked."

        attempts_left = MAX_FAILED_ATTEMPTS - user_data['failed_attempts']
        return f"Login Failed: Invalid password. You have {attempts_left} attempt(s) left."

def generate_random_strong_password():

    global attacker_dictionary
    print("Generating a strong password...")
    while True:
        alphabet = secrets.token_urlsafe(16)
        password = ''.join(secrets.choice(alphabet) for i in range(16))
        guess_count = calculate_password_complexity(password, attacker_dictionary)
        strength = get_strength_rating(guess_count)

        if strength in ["Strong", "Very Strong"]:
            print("...Success! Found a strong password.")
            return password



In [8]:
if __name__ == '__main__':

    uploaded = files.upload()

    if not uploaded:
        print("\n--- ERROR ---")
        print("No file was uploaded. Please re-run the cell and select the file.")
    else:
        actual_filename = list(uploaded.keys())[0]

        attacker_dictionary = load_ranked_dictionary(actual_filename, MAX_DICT_SIZE)

        print("\n--- Create Your Account ---")

        while True:
            username = input("Enter a new username: ")
            password = getpass.getpass("Enter a new password: ")

            result = sign_up(username, password)
            print(result)

            if result.startswith("✅ Success"):
                break
            else:
                print("Please try again.")

        print("\n--- Please Login ---")

        while True:
            username = input("Enter your username: ")
            password = getpass.getpass("Enter your password: ")

            result = login(username, password)
            print(result)

            if result.startswith("✅ Login Successful"):
                print("\nWelcome! You are now logged in.")
                break

            elif "Account is locked" in result:
                print("Please contact support to unlock your account.")
                break

            else:
                print("Please try again.")

Please upload your password dataset file


Saving production.tsv to production (5).tsv
Loading ranked dictionary from production (5).tsv...
Loaded 50000 unique passwords into ranked dictionary.

--- Create Your Account ---
Enter a new username: nahas.m
Enter a new password: ··········
✅ Success: Account 'nahas.m' created. (Strength: Very Strong)

--- Please Login ---
Enter your username: nahas.m
Enter your password: ··········
Login Failed: Invalid password. You have 2 attempt(s) left.
Please try again.
Enter your username: nahas.m
Enter your password: ··········
Login Failed: Invalid password. You have 1 attempt(s) left.
Please try again.
Enter your username: nahas.m
Enter your password: ··········
Login Failed: Invalid password. Your account is now locked.
Please try again.
Enter your username: nahas.m
Enter your password: ··········
Login Failed: Account is locked due to too many failed attempts.
Please contact support to unlock your account.
